In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').master('local[4]').getOrCreate()

22/06/14 10:44:11 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:44:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:44:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:44:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:44:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:44:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:44:13 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('../data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


### Creating Feature Vector From Data Columns

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
# we are only interested in last 5 numeric columns
assembler = VectorAssembler(
    inputCols=[
        'Avg Session Length',
         'Time on App',
         'Time on Website',
         'Length of Membership',
         'Yearly Amount Spent'],
    outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
for item in output.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005
[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005]


In [13]:
row = output.select('features').head(1)[0]

In [14]:
row.asDict()

{'features': DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511])}

In [15]:
final_data = output.select('features', 'Yearly Amount Spent')

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

### Create Train Test Split

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean| 501.53818674985195|
| stddev|  82.17885326068769|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 494.12435844564754|
| stddev|  72.18689079023311|
|    min|  282.4712457199145|
|    max|  725.5848140556806|
+-------+-------------------+



### Train Linear Regression Model

In [20]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [21]:
lr_model = lr.fit(train_data)

22/06/14 10:44:28 WARN Instrumentation: [e85402ec] regParam is zero, which might cause numerical instability and overfitting.


In [22]:
test_results = lr_model.evaluate(test_data)

In [23]:
# residual is difference between predicted and actual values
test_results.residuals.show()

/opt/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+
|           residuals|
+--------------------+
|-1.00612851383630...|
|-4.20641299569979...|
|-1.33013600134290...|
|-5.22959453519433...|
|-5.85487214266322...|
|-1.04591890703886...|
|-6.25277607468888...|
|-1.87583282240666...|
|4.376943252282217...|
|-3.75166564481332...|
|-4.14956957683898...|
|-5.45696821063756...|
|-2.84217094304040...|
|-4.26325641456060...|
|1.932676241267472...|
|-8.24229573481716...|
|-1.13686837721616...|
|-1.00044417195022...|
|-2.21689333557151...|
|-5.45696821063756...|
+--------------------+
only showing top 20 rows



In [24]:
# root mean squared error
test_results.rootMeanSquaredError

4.385478667992488e-12

In [25]:
# r^2
test_results.r2

1.0

In [26]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



### Deploying Model

In [27]:
unlabeled_data = test_data.select('features')

In [28]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3662121671876...|
|[31.5147378578019...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8293464559211...|
|[31.8512531286083...|
+--------------------+
only showing top 20 rows



In [29]:
predictions = lr_model.transform(unlabeled_data)

In [30]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 282.4712457199246|
|[30.5743636841713...|442.06441375806986|
|[31.1239743499119...| 486.9470538397791|
|[31.1280900496166...| 557.2526867470599|
|[31.1695067987115...|427.35653080229866|
|[31.2681042107507...| 423.4705331738344|
|[31.3091926408918...| 432.7207178399399|
|[31.3662121671876...| 430.5888825564868|
|[31.5147378578019...|489.81248799645704|
|[31.5761319713222...| 541.2265839893321|
|[31.6098395733896...| 444.5455496511123|
|[31.6253601348306...|376.33690075692965|
|[31.6548096756927...| 475.2634237275488|
|[31.6610498227460...| 416.3583535799051|
|[31.7207699002873...|  538.774933478021|
|[31.7242025238451...|503.38788728796874|
|[31.7656188210424...|496.55408163560725|
|[31.8124825597242...| 392.8103449838072|
|[31.8293464559211...| 385.1523379879772|
|[31.8512531286083...|472.99224666680385|
+--------------------+------------